In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
from src.models.EMOS_global.EMOS_global_load_score import *
from src.models.EMOS_local.EMOS_local_load_score import *
from src.models.EMOS_global.EMOS_global_load_model import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *
from src.models.EMOS_local.EMOS_local_train import *

2023-06-01 10:52:12.593213: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-01 10:52:12.593234: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 1. Load Scores

In [24]:
EMOS_local_t2m_scores = EMOS_local_load_score("t2m")
EMOS_global_t2m_scores = EMOS_global_load_score("t2m")

<IPython.core.display.Javascript object>

In [25]:
len(EMOS_local_t2m_scores)

31

<IPython.core.display.Javascript object>

In [26]:
EMOS_local_t2m_scores[0].shape

(120, 130)

<IPython.core.display.Javascript object>

In [27]:
EMOS_local_t2m_scores[0]

array([[3.50032588e+00, 6.44441549e+02, 8.21504443e+01, ...,
        1.35585223e+02, 1.97968435e+02, 6.48960187e+00],
       [2.62426196e+02, 1.67820612e+02, 9.52243915e+01, ...,
        5.36963152e+01, 4.62368605e+02, 3.82714911e+02],
       [1.16907437e+02, 4.75848548e+02, 1.53578152e+02, ...,
        4.90139212e+02, 4.90578984e+01, 4.30608516e+02],
       ...,
       [1.75768962e+02, 1.04779946e+02, 4.38938723e-01, ...,
        1.33896160e+02, 3.37669101e+02, 5.56495298e+02],
       [2.22584369e+01, 2.46425516e+01, 3.06345613e+02, ...,
        3.05350471e+02, 5.87215091e+02, 7.02386294e+02],
       [1.53914988e+02, 5.83249297e+02, 5.16467201e+02, ...,
        1.31116079e+02, 1.18435279e+02, 3.28425379e+02]])

<IPython.core.display.Javascript object>

In [28]:
for i in range(len(EMOS_local_t2m_scores)):
    print(np.mean(EMOS_local_t2m_scores[i]))

327.2017108996968
328.0493000664214
332.48294152838247
330.6740324498626
326.73660801611754
326.70713238555567
329.32984385920594
323.13521994302386
325.1597666886211
328.6905720557139
326.0973693368299
328.52279513621494
321.9730010965976
329.28202453296285
328.9977007887049
322.3121892305651
324.76438635533987
332.51399195103966
329.1149755829886
325.0247790620081
325.27671443053555
327.84375494602887
328.1380074302177
325.2698518401066
326.17470938007926
329.4066426815335
328.1277118818608
324.0014038933928
324.11201905947763
328.7127176491239
0.0


<IPython.core.display.Javascript object>

In [16]:
for i in range(len(EMOS_global_t2m_scores)):
    print(np.median(EMOS_global_t2m_scores[i]))

0.3579158720675525
0.4294854308486648
0.43580610345285276
0.46634451031549323
0.4805410959290852
0.5147794461997237
0.5319366754911081
0.5655238974572119
0.5731764403980474
0.5984597704946297
0.6204348943894081
0.6523478367241466
0.6587426287984816
0.6819800091642969
0.7016747915923386
0.7306790653307432
0.7344800723014707
0.7602097936026542
0.7728042089225389
0.8031066091325454
0.8045481476610289
0.838901271578145
0.8489514579129032
0.8728372701922087
0.8741058417649076
0.8986773938169144
0.9177029102092923
0.9431298210736898
0.9401560790901858
0.969711973380655
0.980268295618246


<IPython.core.display.Javascript object>

### 2. Hyperparameter tuning:

In [22]:
var_num = 2
lead_time = 15
batch_size = 32
epochs = 30
lr = 0.01
validation_split = 0.2
optimizer = "Adam"

<IPython.core.display.Javascript object>

In [23]:
EMOS_local_train(
    var_num,
    lead_time,
    batch_size=batch_size,
    epochs=epochs,
    lr=lr,
    validation_split=validation_split,
    optimizer=optimizer,
    save=False,
)

2023-06-01 10:55:43.260695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-01 10:55:43.260814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-01 10:55:43.260906: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-01 10:55:43.260996: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file:

Best validation score = 0.538581371307373
Best validation score = 0.5404307246208191
Best validation score = 0.5648877024650574
Best validation score = 0.5633873343467712
Best validation score = 0.563684344291687
Best validation score = 0.5657873153686523


KeyboardInterrupt: 

<IPython.core.display.Javascript object>

### 3. Trying to load models: